In [ ]:
#@title Install Libraries
!pip install SpeechRecognition==3.8.1
!pip install pydub==0.24.1
!pip install pytube3
!pip install pytube

In [ ]:
#@title Download Video

url = "video url here" #@param {type:"string"}
# http://www.lingoes.net/en/translator/langcode.htm
Type = "Subtitles" #@param ["Transcription", "Subtitles"]
Language = "en-US" #@param ["en-US", "it-IT", "es-ES", "de", "fr"]
Language = str(Language)
import os
import sys
import time # for delay
import pydub
import pytube
import datetime
import speech_recognition as sr
from os import listdir
from pydub import AudioSegment

if not os.path.exists('input'):
    os.makedirs('input')
if not os.path.exists('output'):
    os.makedirs('output')

input_folder = os.path.join(os.getcwd(), "input")

file_list = os.listdir(input_folder)

try:
    ytd = pytube.YouTube(url)
    #print('\nVideo Title:')
    name = ytd.title
    print(ytd.title)
    for l in file_list:
      if name not in l and l[0] != ".":
        os.remove(os.path.join(path_input, l))

    # Download only if not already downlaoded
    name = name + ".mp4"
    
    if name not in file_list:
      print("\n..Downloading..")
      ytd = pytube.YouTube(url).streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(input_folder)
    
    print('\n# Done #\n')

except pytube.exceptions.RegexMatchError as e:
    print('Error : {}'.format(e))

path_input = os.getcwd() + os.sep +  'input'
path_output = os.getcwd() + os.sep +  'output'


# pydub.AudioSegment.ffmpeg = "C:/ffmpeg"

list_v = os.listdir(path_input)
if len(list_v) < 1 :
    print('No files to transcribe!')
    sys.exit()

file_name = list_v[0]

# get audiofile into usable form
audio = AudioSegment.from_file(os.path.join(path_input, str(file_name)))
offset = 3000 # 15000 == 15 seconds -- calculate in millisecond

# initialize recognizer
r = sr.Recognizer()

if Type == "Transcription":
  print(f"Creating Transcription - {Language}\n")
  parts = 1 + (len(audio) // offset)
  for j in range(parts):
      # get intervals of 15 from audio data
      t1 = j * offset
      t2 = t1 + offset
      audio_segment = audio[t1:t2 + 1000]

      audio_segment.export(os.path.join(path_input, 'temporary.wav') , format="wav")

      start = str(datetime.timedelta(seconds=t1 / 1000))
      end = str(datetime.timedelta(seconds=t2 / 1000))
      print('Part {} of {} \t [ {}s : {}s ]'.format(j+1,parts, start,end ))
      try:
          with sr.AudioFile( os.path.join(path_input, 'temporary.wav')) as source:
              aux = r.record(source)  # read the entire audio file

          # call to google APIs
          data = r.recognize_google(aux, language=Language, show_all=False) # English

          file_object = open(os.path.join(path_output, file_name) + '_google.txt' , 'a')
          file_object.write('\n[ ' + str(datetime.timedelta(seconds=t1 / 1000)) + 's : ' + str(datetime.timedelta(seconds=t2 / 1000)) + 's ]\n')
          if str(data)[-1] == ' ':
              file_object.write(str(data))
          else:
              file_object.write(str(data) + ' ')
          file_object.close()

      except sr.UnknownValueError:
          print("Google Speech Recognition could not understand audio")
      except sr.RequestError as e:
          print("Could not request results from Google Speech Recognition service; {0}".format(e))
      #delete aux files of 15 seconds
      os.remove(os.path.join(path_input, 'temporary.wav'))

if Type == "Subtitles":
  print(f"Creating Subtitles - {Language} \n")
  parts = 1 + (len(audio) // offset)
  for j in range(parts):
      # get intervals of 15 from audio data
      t1 = j * offset
      t2 = t1 + offset
      audio_segment = audio[t1:t2 + 1000]

      audio_segment.export(os.path.join(
          path_input, 'temporary.wav'), format="wav")

      start = str(datetime.timedelta(seconds=t1 / 1000))
      end = str(datetime.timedelta(seconds=t2 / 1000))
      print('Part {} of {} \t [ {}s : {}s ]'.format(j+1, parts, start, end))
      try:
          with sr.AudioFile(os.path.join(path_input, 'temporary.wav')) as source:
              aux = r.record(source)  # read the entire audio file

          # call to google APIs
          data = r.recognize_google(
              aux, language=Language, show_all=False)  # English

          file_object = open(os.path.join(
              path_output, file_name) + '_google.srt', 'a')
          #file_object.write('\n[ ' + str(datetime.timedelta(seconds=t1 / 1000)) + 's : ' + str(datetime.timedelta(seconds=t2 / 1000)) + 's ]\n')
          if str(data)[-1] == ' ':
              file_object.write('\n\n' + str(j) + '\n' +
                                '{} --> {}'.format(start, end) + '\n' + str(data))
          else:
              file_object.write(
                  '\n\n' + str(j) + '\n' + '{} --> {}'.format(start, end) + '\n' + str(data) + ' ')
          file_object.close()

      except sr.UnknownValueError:
          print("Google Speech Recognition could not understand audio")
      except sr.RequestError as e:
          print(
              "Could not request results from Google Speech Recognition service; {0}".format(e))
      # delete aux files of 15 seconds
      os.remove(os.path.join(path_input, 'temporary.wav'))
# Last step
print('\n# Done # ')